In [1]:
import requests
from datetime import datetime
import pandas as pd
import re 
import csv
import json
import time
import mysql.connector
import pymysql, os
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine


In [2]:
now = datetime.today()

In [3]:
list_province_in_thai = []
list_district_in_thai = []
list_subdistrict_in_thai = []
label = pd.read_excel('../code/geocodelabel.xlsx', index_col=None)
for i in range(len(label)):
    list_province_in_thai.append(label['name_th'][i])
    list_district_in_thai.append(label['district_name_th'][i])
    list_subdistrict_in_thai.append(label['subdistrict_name_th'][i])

# clear dup
list_province_in_thai = list(dict.fromkeys(list_province_in_thai))
list_district_in_thai = list(dict.fromkeys(list_district_in_thai))
list_subdistrict_in_thai = list(dict.fromkeys(list_subdistrict_in_thai))


In [4]:
def longdoAddress(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    #print("------------------------------------------")
    #print(payload)
    #print("------------------------------------------")
    province = removeWrongkeyword(payload['province'])
    district = removeWrongkeyword(payload['district'])
    subdistrict = removeWrongkeyword(payload['subdistrict'])
    try:
        postcode = payload['postcode']
        
    except :
        postcode = ""
    
    return province,district,subdistrict,postcode

In [5]:
def removeWrongkeyword(word):
    payload_word = word
    if(re.findall(r"\จ+[.]",word)):
        payload_word = re.sub(r"\จ+[.]","จังหวัด",word)
    elif(re.findall(r"\อ+[.]",word)):
        payload_word = re.sub(r"\อ+[.]","อำเภอ",word)
    elif(re.findall(r"\ต+[.]",word)):
        payload_word = re.sub(r"\ต+[.]","ตำบล",word)
    return payload_word
        

In [6]:
def extracteachDetail(word):
    payload_word = word
    if(re.findall(r"\จังหวัด",payload_word)):
        payload_word = re.sub("จังหวัด","",payload_word)
    elif(re.findall(r"\อำเภอ",payload_word)):
        payload_word = re.sub("อำเภอ","",payload_word)
    elif(re.findall(r"\ตำบล",payload_word)):
        payload_word = re.sub("ตำบล","",payload_word)
    elif(re.findall(r"\เขต",payload_word)):
        payload_word = re.sub("เขต","",payload_word)
    elif(re.findall(r"\แขวง",payload_word)):
        payload_word = re.sub("แขวง","",payload_word)
        
    return payload_word

In [7]:
def getdataFromAPI():
    url = "https://corporate.lotuss.com/location/json-data/"
    response = requests.get(url)
    payload = response.json()
    with open(f"../trash/Lotus{str(now.date())}.json", 'w') as f:
        json.dump(payload, f)

In [8]:
def getAddressDetail(address,lat,lng):
    
    str_address =  re.sub(r'["]',"",address)

    str_address = str_address.replace("  "," ")
    
    #remove newline 
    if("\n" in str_address) :
        str_address = re.sub('\r','',str_address)
        str_address = re.sub('\n',' ',str_address)
    
    temp_fix = str_address.split()
    
    payload_address = ""
    
    #fix address info
    if(len(temp_fix) == 1):
        #print("case anomaly")
        province,district,subdistrict,postcode = longdoAddress(lat,lng)
        payload_address = "-"
    
    else :
        #case postcode stick with province
        if(re.findall("[ก-ฮ]", temp_fix[-1]) and re.findall("[0-9]", temp_fix[-1])):
            split_province_post = temp_fix[-1]
            numbers = []
            for interger in split_province_post :
                if interger.isdigit():
                    numbers.append(interger)
                    
            temp_post = ""
            for post in numbers:
                split_province_post = split_province_post.replace(post,"")
                temp_post = temp_post + post
                
            # special case for bangkok
            if(split_province_post == "กทม."):
                split_province_post = "จังหวัดกรุงเทพมหานคร"
            elif re.findall("จ+[.]", split_province_post):
                split_province_post = split_province_post.replace("จ.","จังหวัด")
            temp_fix.remove(temp_fix[-1])
            temp_fix.append(split_province_post)
            temp_fix.append(temp_post)
            #print(temp_fix[-1])
        
        # next case postcode missing
        if re.findall("[ก-ฮ]", temp_fix[-1]):
            _,_,_,postcode = longdoAddress(lat,lng)
            #print("----------------------")
            if(postcode == ""):
                postcode = "-"
            else :
                temp_fix.append(postcode)
        
        # remove keyword
    
            # province 
        if(not re.findall("[0-9]", temp_fix[-1])) :

            temp_fix[-1] = removeWrongkeyword(temp_fix[-1])
            temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
            temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
            
            province = extracteachDetail(temp_fix[-1])
            district = extracteachDetail(temp_fix[-2])
            subdistrict = extracteachDetail(temp_fix[-3])
            
        else :
            
            temp_fix[-2] = removeWrongkeyword(temp_fix[-2])
            temp_fix[-3] = removeWrongkeyword(temp_fix[-3])
            temp_fix[-4] = removeWrongkeyword(temp_fix[-4])
            
            postcode = extracteachDetail(temp_fix[-1])
            province = extracteachDetail(temp_fix[-2])
            district = extracteachDetail(temp_fix[-3])
            subdistrict = extracteachDetail(temp_fix[-4])

        for add in temp_fix :
        
            payload_address = payload_address + add + " "
    '''
    print(payload_address)
    print("province: " + province)  
    print("district: " + district)  
    print("subdistrict: " + subdistrict)  
    print("postcode: " + postcode)  
    print("--------------------------------------------")
    '''
    
    return payload_address,province,district,subdistrict,postcode
            


In [9]:
def getStoretypename(id):
    type_store = {
        "stores-lotuss-extra": "Hypermarket",
        "stores-lotuss-market": "Supermarket",
        "stores-lotuss-express": "Convenience Stores",

    }
    store_name = {
        "stores-lotuss-extra": "Lotus Extra",
        "stores-lotuss-market": "Lotus market",
        "stores-lotuss-express": "Lotus's go fresh",

    }
    return type_store.get(id) , store_name.get(id)


In [10]:
'''
dict_keys(['address', 'id', 'latitude', 'line_url', 'longitude', 'name', 'opening_hours', 'phone_number', 'type', 'type_name'])
'''

"\ndict_keys(['address', 'id', 'latitude', 'line_url', 'longitude', 'name', 'opening_hours', 'phone_number', 'type', 'type_name'])\n"

In [11]:
getdataFromAPI()
f = open(f'../trash/Lotus{str(now.date())}.json')
payload = json.load(f)
length = len(payload['data'])

In [12]:
list_Lotus_Extra = []
list_Lotus_market = []
list_Lotus_go_fresh = []
list_lotus = []
for i in range (length):
    name = payload['data'][i]['name']
    store_id = payload['data'][i]['id']
    lat = payload['data'][i]['latitude']
    lng = payload['data'][i]['longitude']
    address = payload['data'][i]['address']
    type_store = payload['data'][i]['type']
    store_type,store_type_name = getStoretypename(type_store)

    if((lat > 0) or (lng > 0)):
        #print("index of: " + str(i))
        payload_address,province,district,subdistrict,postcode = getAddressDetail(address,lat,lng)
    
        dict_payload = { "name": name,
                        "store_id" : store_id,
                        "name_store" : store_type_name, 
                        "store_type" : store_type ,
                        "address" : payload_address ,
                        "lat" : lat,
                        "lng" : lng,
                        "subdistrict": subdistrict ,
                        "district": district,
                        "province": province,
        }
        
        if(store_type == "Hypermarket") :
            list_Lotus_Extra.append(dict_payload)
        elif(store_type == "Supermarket") :
            list_Lotus_market.append(dict_payload)
        elif(store_type == "Convenience Stores") :
            list_Lotus_go_fresh.append(dict_payload)



In [13]:
with open(f"../trash/Lotus_Extra{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_Lotus_Extra, outfile)
with open(f"../trash/Lotus_market{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_Lotus_market, outfile)
with open(f"../trash/Lotusgofresh{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_Lotus_go_fresh, outfile)

In [14]:
df_json =  pd.read_json(f"../trash/Lotus_market{str(now.date())}.json")
df_json.to_excel(f"../excel/Lotus/LotusMarket/Lotus_market{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Lotus/LotusMarket/Lotus_market{str(now.date())}.csv")

In [15]:

df_json =  pd.read_json(f"../trash/Lotus_Extra{str(now.date())}.json")
df_json.to_excel(f"../excel/Lotus/LotusExtra/Lotus_Extra{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Lotus/LotusExtra/Lotus_Extra{str(now.date())}.csv")


In [16]:

df_json =  pd.read_json(f"../trash/Lotusgofresh{str(now.date())}.json")
df_json.to_excel(f"../excel/Lotus/LotusGoFresh/Lotusgofresh{str(now.date())}.xlsx")
df_json.to_csv(f"../csv/Lotus/LotusGoFresh/Lotusgofresh{str(now.date())}.csv")


In [ ]:
#lotus
le = pd.read_csv(f'../csv/Lotus/LotusExtra/Lotus_Extra{str(now.date())}.csv', header = 0)
lm = pd.read_csv(f'../csv/Lotus/LotusMarket/Lotus_market{str(now.date())}.csv', header = 0)
lg = pd.read_csv(f'../csv/Lotus/LotusGoFresh/Lotusgofresh{str(now.date())}.csv', header = 0)
engine = create_engine('mysql://root:@localhost:3306/lotusextra')
with engine.connect() as conn, conn.begin():
    le.to_sql(f'Lotus_Extra{str(now.date())}', conn, if_exists='append', index=False)
conn.close()
engine = create_engine('mysql://root:@localhost:3306/lotusmarket')
with engine.connect() as conn, conn.begin():
    lm.to_sql(f'Lotus_market{str(now.date())}', conn, if_exists='append', index=False)
conn.close()
engine = create_engine('mysql://root:@localhost:3306/lotusgofresh')
with engine.connect() as conn, conn.begin():
    lg.to_sql(f'Lotusgofresh{str(now.date())}', conn, if_exists='append', index=False)
conn.close()

In [ ]:
#updateGIS
#คิดไม่ออกโว้ยยยยยยยยยยยยยยยยยยย